In [22]:
# if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget

In [23]:
import os
import sys
import wget
import tarfile
import zipfile
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb

In [24]:
DDIR = "data"
UDDIR = "user_data"
RESDIR = "prediction_result"

# Load data (Only once)

In [34]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [35]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [36]:
# myunzip(train_fname)
# myunzip(test_fname)

# Data Exploration

In [ ]:
# read train data
train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# tag
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [ ]:
# read test data
test_click_log = pd.read_csv(f"{DD}")